In [92]:
import pandas as pd
import numpy as np
import math

ratings_path = "./ml-latest-small/ratings.csv"
movies_path = "./ml-latest-small/movies.csv"
tags_path = "./ml-latest-small/tags.csv"

In [93]:
Ratings = pd.read_csv(ratings_path)
Movies = pd.read_csv(movies_path)
Tags = pd.read_csv(tags_path)

In [94]:
MeanUserRating = Ratings.groupby(['userId'], as_index = False, sort = False).mean().rename(columns = {'rating': 'rating_mean'})[['userId','rating_mean']]
Ratings = pd.merge(Ratings, MeanUserRating, on = 'userId', how = 'left', sort = False)
Ratings['rating_adjusted'] = Ratings['rating'] - Ratings['rating_mean']
Ratings['mean_skewness'] = (Ratings['rating_adjusted'])/(5-Ratings['rating_mean'])*(Ratings['rating_adjusted']>0).astype(float) - (Ratings['rating_adjusted'])/(Ratings['rating_mean']-0.5)*(Ratings['rating_adjusted']<0).astype(float)
Ratings['rating_unskewed'] = Ratings['rating_adjusted'] * np.sqrt(1+(Ratings['mean_skewness']**2)*2) 
Ratings = Ratings.drop(columns=['rating', 'rating_mean','timestamp'])

In [105]:
distinct_users = np.unique(Ratings['userId'])
similar_users = pd.DataFrame()

for user1 in distinct_users[:10]:
    user1_data = Ratings[Ratings['userId']==user1]
    #user1_mean = user1_data['rating_mean'].iloc[0]
    #user1_data = user1_data.drop(columns=['rating_mean'])
    user1_data = user1_data.rename(columns={'rating_adjusted':'rating_adjusted1'})
    user1_data = user1_data.rename(columns={'rating_unskewed':'rating_unskewed1'})
    user1_data = user1_data.rename(columns={'userId':'userId1'}) 
    #user1_movies = np.unique(user1_data['movieId'])
    user1_vector_length = np.sqrt(np.sum(np.square(user1_data['rating_adjusted1']), axis=0))
    
    for user2 in distinct_users[:10]:
        user2_data = Ratings[Ratings['userId']==user2]
        #user2_mean = user2_data['rating_mean'].iloc[0]
        #user2_data = user2_data.drop(columns=['rating_mean'])
        user2_data = user2_data.rename(columns={'rating_adjusted':'rating_adjusted2'})
        user2_data = user2_data.rename(columns={'rating_unskewed':'rating_unskewed2'})
        user2_data = user2_data.rename(columns={'userId':'userId2'}) 
        #user2_movies = np.unique(user2_data['movieId'])
        user2_vector_length = np.sqrt(np.sum(np.square(user2_data['rating_adjusted2']), axis=0))
        
        user_data = pd.merge(user1_data,user2_data[['rating_adjusted2','userId2','movieId', 'rating_unskewed2']], on = 'movieId', how = 'inner', sort = False)
        user_data['vector_product']=(user_data['rating_adjusted1']*user_data['rating_adjusted2'])
        user_data['unskewed_product']=(user_data['rating_unskewed1']*user_data['rating_unskewed2'])
        #records_in_common = user_data.shape[0]
        user_data = user_data.groupby(['userId1','userId2'], as_index = False, sort = False)['vector_product', 'unskewed_product'].agg('sum')
        user_data['pearson'] = user_data['vector_product']/(user1_vector_length*user2_vector_length)
        user_data['unskewed_pearson'] = user_data['unskewed_product']/(user1_vector_length*user2_vector_length)
        user_data = user_data.drop(columns=['vector_product', 'unskewed_product'])
        similar_users = similar_users.append(user_data, ignore_index=True)
        
similar_users = similar_users[similar_users['pearson']<0.99999]

results = similar_users.sort_values(['unskewed_pearson'], ascending=False).head(20)

In [106]:
results

,userId1,userId2,pearson,unskewed_pearson
28,4,7,0.057519,0.150997
52,7,4,0.057519,0.150997
84,10,9,0.043256,0.136108
76,9,10,0.043256,0.136108
51,7,3,0.054900,0.134764
18,3,7,0.054900,0.134764
65,8,9,0.039032,0.132273
74,9,8,0.039032,0.132273
60,8,4,0.055430,0.126888
29,4,8,0.055430,0.126888


In [ ]:
user_dot_adj_rating_all = pd.DataFrame()

distinct_movies = np.unique(Ratings['movieId'])

j=1
for movie in distinct_movies:

    user_data_append_movie = user_data_all[user_data_all['movieId']==movie]
    user_dot_adj_rating = pd.merge(Ratings,user_data_append_movie[['pearson','userId2','userId1']], how = 'inner',left_on='userId', right_on='userId2', sort = False)

    if j%200==0:
        print (j , "out of ", len(distinct_movies))

    user_dot_adj_rating1 = user_dot_adj_rating[user_dot_adj_rating['movieId']==movie]

    if len(np.unique(user_dot_adj_rating1['userId'])) >= 2:
        user_dot_adj_rating1['wt_rating'] = user_dot_adj_rating1['pearson']*user_dot_adj_rating1['rating_adjusted']

    user_dot_adj_rating1['pearson_abs'] = user_dot_adj_rating1['pearson'].abs()
    user_dot_adj_rating1 = user_dot_adj_rating1.groupby(['userId1'], as_index = False, sort = False).sum()[['userId1','wt_rating','pearson_abs']]
    user_dot_adj_rating1['Rating'] = (user_dot_adj_rating1['wt_rating']/user_dot_adj_rating1['pearson_abs'])+user1_mean
    user_dot_adj_rating1['movieId'] = movie
    user_dot_adj_rating1 = user_dot_adj_rating1.drop(['wt_rating', 'pearson_abs'], axis=1)

    user_dot_adj_rating_all = user_dot_adj_rating_all.append(user_dot_adj_rating1, ignore_index=True)

    j=j+1

    user_dot_adj_rating_all = user_dot_adj_rating_all.sort(['Rating'], ascending=False)